In [42]:
import pandas as pd
import numpy as np
import tqdm
import glob
import sys
# from enrichment_scores import score_abid_pval_and_enrichment
import os

In [43]:
def score_abid_pval_and_enrichment(df, program, nbins, nbackground, obs_cut = None):
  if obs_cut is None:
    obs_avg = df.mean(axis=0)
    n_items = int(np.round(len(obs_avg) / (nbins - 1)))
    obs_cut = (obs_avg.rank(method='min') // n_items).astype("int")


  program_mean = df.T.loc[program].mean(axis=0)

  background = []
  for i in range(nbackground):
      background_genes = []

      for cut in obs_cut.loc[program]:
          r_genes = set(obs_cut[obs_cut == cut].index) - set(program)
          r_genes = np.array(list(r_genes))
          np.random.shuffle(r_genes)
          # uses full r_genes if ctrl_size > len(r_genes)
          background_genes.append(r_genes[0])

      # get the mean expression of the background genes
      
      background.append(df.T.loc[background_genes].mean(axis=0))
      # print(background)
      # break

  background = pd.DataFrame(background).T
  print("background")
  print(background)
  result_enrichment = program_mean - background.mean(axis=1)
  
  result_pval = (program_mean < background[0]).astype(int)
  for i in range(1, len(background.columns)):
      result_pval = result_pval + (program_mean < background[i]).astype(int)

  pval = result_pval / len(background.columns)

  return pval, result_enrichment

In [44]:
option_idx =  1

options_df = pd.read_csv("options_ot_updated.csv", index_col = 0)
fno_idx = options_df.loc[option_idx, "fno"]
pno_idx = options_df.loc[option_idx, "pno"]

# load files
files = np.sort(list(set(glob.glob("./adata*.csv")) \
             - set(glob.glob("./*spatial*"))))

# load programs
sheet_url = "https://docs.google.com/spreadsheets/d/1N5Hlo6ASSnrpV2GoZHSKPhRX8j_oFOYSFhcgyEgrlpo/edit#gid=0"
sheet_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
programs_df = pd.read_excel("ot_gene_lists.xlsx", sheet_name="programs")
programs = np.sort(programs_df["Program"].unique())

print("programs:")
print(programs)

fname = files[fno_idx]
pname = programs[pno_idx]

print('fname', fname)
print('pname', pname)

print(fname.split("/")[-1], pname)

df = pd.read_csv(fname, index_col=0)
df

programs:
['Apoptosis' 'Autophagy' 'CM_Ankrd1' 'CM_Myh6' 'CM_Xirp2'
 'Cardiac_progenitor_cell_lingeage_commitment_Nkx2-5_CPC'
 'Cardiac_progenitor_cell_lingeage_commitment_ls1_CPC'
 'Cardiomyocyte_maturation_CIRCRESAHA' 'Cardiomyocyte_maturation_Uosaki'
 'Cell_cycle' 'Fib1' 'Fib2' 'Fib3' 'Fib4' 'HLHS_DEGs' 'Immune_Arg1'
 'Immune_DC' 'Immune_ISG' 'Immune_Lyve1' 'Immune_Mono' 'Immune_NSG'
 'Immune_Retnlg' 'Immune_Sigf' 'Ion_channel' 'Myeloid_DCs_FLT3_ITGAX'
 'Myeloid_LYVE_FOLR_Macrophages' 'Myeloid_LYVE_PLTP_Macrophages'
 'Myeloid_Monocyte_CCL18' 'Myeloid_Monocyte_SPP1' 'Senescence'
 'Stromal_Cxcl5' 'Stromal_Dkk3' 'Stromal_EC_Cyyr1' 'Stromal_EC_Fbln5'
 'Stromal_EC_Kit' 'Stromal_EC_Npr3' 'Stromal_EC_ccl21a' 'Stromal_Gsn'
 'Stromal_Hsd11b1' 'Stromal_Myh11' 'Stromal_Pi16' 'Stromal_Postn'
 'Stromal_Rep' 'Stromal_SMC' 'Tead_SCENIC_Yang_GD_fibroblastpaper'
 'Unfolded_protein_response' 'Yap5SA_Targets_CM_dropseq' 'bz1_king'
 'bz2_king' 'iz_king' 'rz_king' 'vCM1' 'vCM2' 'vCM3' 'vCM4' 'vCM5']
fna

,Trmt2a,Runx2,Dusp7,Arl2bp,Arfgef1,Slc25a42,Arhgap17,Tec,B230322F03Rik,Smpd3,...,Cbx3,Fhod1,Prkce,Nudt16l1,Ptprm,Tcn2,Fam3a,Gss,Dtx3,Rhou
AAACAAGTATCTCCCA-1,1.566328,0.000000,0.564067,1.841287,1.394070,0.000000,0.922516,0.922516,0.000000,0.564067,...,1.566328,0.000000,0.922516,0.564067,1.185835,1.841287,1.394070,0.564067,1.394070,0.000000
AAACAATCTACTAGCA-1,0.576147,0.576147,0.576147,1.415047,1.415047,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.576147,0.000000,0.576147,0.000000,0.939359,1.205225,0.000000,0.576147,0.576147,0.576147
AAACAGAGCGACTCCT-1,0.534571,0.534571,0.000000,1.895177,1.342045,0.000000,0.881049,0.534571,0.534571,0.000000,...,1.996120,0.534571,0.000000,0.881049,0.000000,1.996120,1.342045,0.534571,0.534571,0.881049
AAACAGCTTTCAGAAG-1,0.702246,0.000000,0.000000,1.399911,0.927199,0.411540,0.000000,0.411540,0.411540,0.000000,...,1.399911,0.000000,0.411540,0.411540,0.702246,1.518195,0.000000,0.702246,0.927199,0.000000
AAACAGGGTCTATATT-1,0.000000,0.000000,0.594280,1.446190,0.964490,0.000000,0.000000,0.594280,0.000000,0.000000,...,0.964490,0.594280,0.964490,0.000000,1.446190,0.594280,0.594280,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTCTAGATACGCT-1,0.000000,0.000000,0.000000,1.355329,1.355329,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.071135,0.000000,0.000000,0.672610,0.000000,1.071135,0.000000,0.000000,0.000000,0.000000
TTGTTTCACATCCAGG-1,0.682209,0.000000,0.000000,0.682209,0.682209,0.000000,0.000000,0.682209,0.000000,0.000000,...,0.000000,0.000000,0.682209,0.000000,0.000000,0.000000,1.084001,0.000000,0.000000,0.000000
TTGTTTCATTAGTCTA-1,0.000000,0.000000,0.722637,0.000000,1.137742,0.000000,0.000000,0.000000,0.000000,0.722637,...,0.000000,0.000000,0.000000,0.000000,1.137742,0.722637,1.137742,0.000000,0.000000,0.000000
TTGTTTCCATACAACT-1,0.000000,0.000000,0.722734,1.656366,1.137870,0.000000,1.137870,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.137870,1.430353,1.137870,0.722734,0.000000,0.000000


In [45]:
nbins = 50
obs_avg = df.mean(axis=0)
obs_avg

Trmt2a     0.504961
Runx2      0.129403
Dusp7      0.553856
Arl2bp     1.110401
Arfgef1    0.817752
             ...   
Tcn2       1.184244
Fam3a      0.542636
Gss        0.319299
Dtx3       0.741575
Rhou       0.482962
Length: 12879, dtype: float64

In [46]:
n_items = int(np.round(len(obs_avg) / (nbins - 1)))
n_items

263

In [47]:
obs_cut = (obs_avg.rank(method='min') // n_items).astype("int")

In [48]:
obs_cut

Trmt2a     28
Runx2       9
Dusp7      29
Arl2bp     41
Arfgef1    36
           ..
Tcn2       41
Fam3a      29
Gss        20
Dtx3       35
Rhou       27
Length: 12879, dtype: int64

In [49]:

results_pval = pd.DataFrame(index = df.index)
results_enrichment = pd.DataFrame(index = df.index)

program_df = programs_df[programs_df["Program"] == pname]
program_genes = list(program_df["Gene"].values)

# print(len(obs_cut))

pval, enrichment = score_abid_pval_and_enrichment(df, program_genes, 50, 1000, obs_cut=obs_cut)
results_pval[pname] = pval
results_enrichment[pname] = enrichment

# os.makedirs(os.path.dirname(f"./results_pval/{fno_idx}_{pno_idx}.csv"), exist_ok=True)
# os.makedirs(os.path.dirname(f"./results_enrichment/{fno_idx}_{pno_idx}.csv"), exist_ok=True)

# results_pval.to_csv(f"./results_pval/{fno_idx}_{pno_idx}.csv")
# results_enrichment.to_csv(f"./results_enrichment/{fno_idx}_{pno_idx}.csv")

background
                         0         1         2         3         4    \
AAACAAGTATCTCCCA-1  1.383862  1.325241  1.319382  1.372085  1.298777   
AAACAATCTACTAGCA-1  1.008476  1.091593  1.085066  1.056127  1.112240   
AAACAGAGCGACTCCT-1  1.283487  1.275708  1.240609  1.270269  1.343239   
AAACAGCTTTCAGAAG-1  0.914313  0.936896  0.883656  0.909130  0.942710   
AAACAGGGTCTATATT-1  0.833316  0.886305  0.837724  0.830778  0.838423   
...                      ...       ...       ...       ...       ...   
TTGTTCTAGATACGCT-1  0.833826  0.935579  0.823013  0.857179  0.858836   
TTGTTTCACATCCAGG-1  0.832173  0.866175  0.849045  0.840345  0.856060   
TTGTTTCATTAGTCTA-1  0.893560  0.999468  0.823860  0.896262  0.922842   
TTGTTTCCATACAACT-1  0.844548  0.941629  0.855664  0.849199  0.913848   
TTGTTTGTGTAAATTC-1  1.045034  1.140482  1.119037  1.170254  1.056182   

                         5         6         7         8         9    ...  \
AAACAAGTATCTCCCA-1  1.293313  1.368304  1.33145

In [50]:
enrichment

AAACAAGTATCTCCCA-1    0.137074
AAACAATCTACTAGCA-1   -0.082099
AAACAGAGCGACTCCT-1    0.190889
AAACAGCTTTCAGAAG-1   -0.142329
AAACAGGGTCTATATT-1   -0.202605
                        ...   
TTGTTCTAGATACGCT-1   -0.197883
TTGTTTCACATCCAGG-1   -0.121685
TTGTTTCATTAGTCTA-1   -0.055921
TTGTTTCCATACAACT-1   -0.186086
TTGTTTGTGTAAATTC-1   -0.026797
Length: 2283, dtype: float64